In [15]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm

In [7]:
#loading our data
chols=pd.read_csv("../Cholesterol_Data/cholesterole_raw.csv")
chols=chols.rename({'Unnamed: 0':'AETID'}, axis=1)
chols=chols.drop(["Series ID"],axis=1)
chols=chols.set_index("AETID",inplace=False)
#
conf=pd.read_csv("../Clinical_Data/Counfounder6_scaled_input_intercept_headers.csv")
conf=conf.set_index("AETID",inplace=False)
conf=conf.drop(["intercept"],axis=1)

In [8]:
#concatenate both dataframes
chols_and_conf = pd.concat([chols, conf], axis=1)
chols_and_conf

,24OH ng/ml,25OH ng/ml,27OH ng/ml,4ß ng/ml,7Keto ng/ml,7aOH ng/ml,7ß ng/ml,Brassicasterol µg/dl,Campestanol µg/dl,Campesterol mg/dl,...,Lathosterol mg/dl,Sitostanol µg/dl,Sitosterol mg/dl,Stigmasterol µg/dl,Age,Smoking,Alcohol,Sex,Levodopa,Group_IPD
AETID,,,,,,,,,,,,,,,,,,,,,
AET0044,71,15.0,121,36.0,334,79,17.70,21.3,3.88,0.360,...,0.238,3.34,0.254,6.67,0.062505,0.0,2.0,1,0,0
AET0380,69,10.0,218,26.8,249,81,14.20,58.5,7.14,0.802,...,0.115,5.44,0.526,9.34,0.890703,1.0,1.0,0,1,0
AET0027,74,14.1,141,17.4,241,76,12.20,27.6,4.12,0.352,...,0.386,3.45,0.275,10.10,-2.185458,0.0,1.0,1,0,0
AET0049,98,16.0,176,32.1,292,148,14.10,60.5,6.21,0.556,...,0.316,5.02,0.440,14.60,1.009017,0.0,2.0,1,0,1
AET0006,63,12.0,137,25.0,130,114,14.00,10.3,3.34,0.213,...,0.491,3.47,0.233,5.80,0.890703,1.0,2.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AET0021,89,13.2,115,35.4,226,128,36.50,18.1,3.05,0.259,...,0.519,2.60,0.131,4.81,-0.292436,1.0,2.0,1,0,0
AET0071,56,16.7,152,26.5,272,89,18.30,21.4,3.41,0.238,...,0.209,2.88,0.174,4.05,0.417447,0.0,3.0,0,0,1
AET0138,58,10.7,190,25.2,286,107,9.61,12.7,3.48,0.175,...,0.306,3.46,0.162,4.79,0.417447,0.0,2.0,1,0,1


In [9]:
chols_and_conf.columns

Index(['24OH ng/ml', '25OH ng/ml', '27OH ng/ml', '4ß ng/ml', '7Keto ng/ml',
       '7aOH ng/ml', '7ß ng/ml', 'Brassicasterol µg/dl', 'Campestanol µg/dl',
       'Campesterol mg/dl', 'Cholestanol mg/dl', 'Cholesterol GC mg/dl',
       'Desmosterol mg/dl', 'DihydroLanesterol µg/dl', 'Lanesterol µg/dl',
       'Lathosterol mg/dl', 'Sitostanol µg/dl', 'Sitosterol mg/dl',
       'Stigmasterol µg/dl', 'Age', 'Smoking', 'Alcohol', 'Sex', 'Levodopa',
       'Group_IPD'],
      dtype='object')

In [10]:
#rename real quick
chols_and_conf.columns=["twentyfouOH","twentyfiveOH","twentysevenOH","fourbeta","sevenKeto","sevenaOH","sevenbeta","Brassicasterol",'Campestanol',"Campesterol","Cholestanol","Cholesterol","Desmosterol","DihydroLanesterol","Lanesterol","Lathosterol","Sitostanol","Sitosterol","Stigmasterol", 'Age', 'Smoking', 'Alcohol', 'Sex', 'Levodopa',
       'Group_IPD']

In [11]:
chols_and_conf.columns
chols_cols=['twentyfouOH', 'twentyfiveOH', 'twentysevenOH', 'fourbeta', 'sevenKeto',
       'sevenaOH', 'sevenbeta', 'Brassicasterol', 'Campestanol', 'Campesterol',
       'Cholestanol', 'Cholesterol', 'Desmosterol', 'DihydroLanesterol',
       'Lanesterol', 'Lathosterol', 'Sitostanol', 'Sitosterol', 'Stigmasterol']

In [8]:
#doing the anova for each cholesterol separately
for mol in chols_cols:
    print(mol)
    m01 = ols(str(mol)+"~"+"Age + Smoking + Alcohol + Sex + Levodopa",data=chols_and_conf).fit()
    m02= ols(str(mol)+"~"+"Age + Smoking + Alcohol + Sex + Levodopa +Group_IPD", data=chols_and_conf).fit()
    anovaResults = anova_lm(m01, m02)
    print(anovaResults)
    print()

twentyfouOH
   df_resid           ssr  df_diff    ss_diff         F    Pr(>F)
0     100.0  62370.328067      0.0        NaN       NaN       NaN
1      99.0  61545.838287      1.0  824.48978  1.326239  0.252249

twentyfiveOH
   df_resid          ssr  df_diff    ss_diff         F    Pr(>F)
0     100.0  6260.516424      0.0        NaN       NaN       NaN
1      99.0  6244.150939      1.0  16.365485  0.259472  0.611617

twentysevenOH
   df_resid            ssr  df_diff      ss_diff         F    Pr(>F)
0     100.0  112278.012543      0.0          NaN       NaN       NaN
1      99.0  111040.152172      1.0  1237.860371  1.103638  0.296027

fourbeta
   df_resid           ssr  df_diff   ss_diff         F    Pr(>F)
0     100.0  15528.746367      0.0       NaN       NaN       NaN
1      99.0  15525.372106      1.0  3.374262  0.021517  0.883679

sevenKeto
   df_resid            ssr  df_diff       ss_diff         F    Pr(>F)
0     100.0  503548.519437      0.0           NaN       NaN       NaN
1  

In [12]:
chols_and_conf.columns[19:]

Index(['Age', 'Smoking', 'Alcohol', 'Sex', 'Levodopa', 'Group_IPD'], dtype='object')

In [13]:
chols_and_conf[chols_and_conf.columns[19:]]

,Age,Smoking,Alcohol,Sex,Levodopa,Group_IPD
AETID,,,,,,
AET0044,0.062505,0.0,2.0,1,0,0
AET0380,0.890703,1.0,1.0,0,1,0
AET0027,-2.185458,0.0,1.0,1,0,0
AET0049,1.009017,0.0,2.0,1,0,1
AET0006,0.890703,1.0,2.0,0,1,1
...,...,...,...,...,...,...
AET0021,-0.292436,1.0,2.0,1,0,0
AET0071,0.417447,0.0,3.0,0,0,1
AET0138,0.417447,0.0,2.0,1,0,1


In [14]:
#MANOVA
manova=MANOVA(chols_and_conf[["Campestanol","Campesterol","Sitostanol","Sitosterol","Stigmasterol"]],chols_and_conf[chols_and_conf.columns[19:]])
print(manova.mv_test())

                 Multivariate linear model
                                                            
------------------------------------------------------------
           x0           Value  Num DF  Den DF F Value Pr > F
------------------------------------------------------------
          Wilks' lambda 0.8204 5.0000 96.0000  4.2023 0.0017
         Pillai's trace 0.1796 5.0000 96.0000  4.2023 0.0017
 Hotelling-Lawley trace 0.2189 5.0000 96.0000  4.2023 0.0017
    Roy's greatest root 0.2189 5.0000 96.0000  4.2023 0.0017
------------------------------------------------------------
                                                            
------------------------------------------------------------
           x1           Value  Num DF  Den DF F Value Pr > F
------------------------------------------------------------
          Wilks' lambda 0.8082 5.0000 96.0000  4.5556 0.0009
         Pillai's trace 0.1918 5.0000 96.0000  4.5556 0.0009
 Hotelling-Lawley trace 0.2373 5.0000 96.0

In [24]:
for chol in chols_and_conf[["Campestanol","Campesterol","Sitostanol","Sitosterol","Stigmasterol"]].columns:
    X = chols_and_conf[chols_and_conf.columns[19:]]
    y = chols_and_conf[[chol]]
    ## fit a OLS model with intercept on TV and Radio
    X = sm.add_constant(X)
    est = sm.OLS(y, X).fit()
    print(est.summary())

                            OLS Regression Results                            
Dep. Variable:            Campestanol   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     1.884
Date:                Fri, 23 Oct 2020   Prob (F-statistic):             0.0909
Time:                        16:25:20   Log-Likelihood:                -186.24
No. Observations:                 106   AIC:                             386.5
Df Residuals:                      99   BIC:                             405.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1698      0.476     10.862      0.0

In [ ]:
#Summary

# all Confounder significant except for Levodopa in multi
# no confounder sigificant in uni ... .
# Group differences for the following chols ( alpha = 5 %):
# - Stigmasterol, Sitosterol, Sitostanol, Campesterol, Campestanol

In [ ]:
anova=anova_lm(chols_and_conf[["Campestanol"]],chols_and_conf[chols_and_conf.columns[19:]])
print(manova.mv_test())

In [24]:
m01 = ols("Sitosterol"+"~"+"Age",data=chols_and_conf).fit()
m02= ols("Sitosterol"+"~"+"Group_IPD", data=chols_and_conf).fit()
anovaResults = anova_lm(m01, m02)
print(anovaResults)
print (m02.summary())

   df_resid       ssr  df_diff   ss_diff    F  Pr(>F)
0     104.0  2.206055      0.0       NaN  NaN     NaN
1     104.0  2.063743     -0.0  0.142313 -inf     NaN
                            OLS Regression Results                            
Dep. Variable:             Sitosterol   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     7.705
Date:                Fri, 09 Oct 2020   Prob (F-statistic):            0.00653
Time:                        16:26:45   Log-Likelihood:                 58.355
No. Observations:                 106   AIC:                            -112.7
Df Residuals:                     104   BIC:                            -107.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t     